In [ ]:
# Generating games data with GPT-4o
import datetime
import logging
import pickle

import pandas as pd

from data_collection.semaphore import set_semaphore
from data_collection.simulate_game import run_games

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

from data_collection.utils import dataset

load_dotenv()

In [ ]:
set_semaphore("gpt-4", 100)

In [ ]:
client = AsyncOpenAI()

In [ ]:
# samples_by_category = {
#     "things": 100,
#     "city": 40,
#     "country": 20,
#     "landmark": 40,
# }
#OVERALL

In [ ]:
# samples_by_category = {
#     "things": 25,
#     "city": 10,
#     "country": 5,
#     "landmark": 10,
# }
# 50 examples

In [ ]:
samples_by_category = {
    "things": 50,
    "city": 0,
    "country": 0,
    "landmark": 0,
}
# 100 examples

In [ ]:
keywords_by_category = pickle.load(open("../keywords_by_category.pkl", "rb"))

In [ ]:
keywords_by_category.keys()

In [ ]:
len(keywords_by_category['things'])

In [ ]:
for key in keywords_by_category:
    keywords_by_category[key] = keywords_by_category[key].sample(samples_by_category[key])
all_dfs = pd.concat(keywords_by_category.values())
keywords_flat = list(all_dfs.keyword)

In [ ]:
keywords_flat

In [ ]:
games = await run_games(client, keywords_flat, include_answer=False)
# games = await run_games(client, ["ice tea"], include_answer=True)

In [ ]:
len([game for game in games if game.win]) / len(games)

In [ ]:
len(dataset)

In [ ]:
dt = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
pickle.dump(dataset, open(f"dataset_{dt}.pkl", "wb"))